In [34]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [37]:
tf.reset_default_graph()
#input placeholder
X = tf.placeholder(tf.float32,[None,784])
X_img = tf.reshape(X, [-1,28,28,1])
Y = tf.placeholder(tf.float32,[None, 10])
keep_prob = tf.placeholder(tf.float32)
learning_rate = 0.01

# Conv layer 1
#L1 ImgIn shape = (?,28,28,1)
with tf.name_scope('ConvLayer1'):
    W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev= 0.01))
    # Conv   ->    (?,28,28,32)
    # Pool   ->    (?,14,14,32)
    L1 = tf.nn.conv2d(X_img, W1, strides= [1,1,1,1], padding= 'SAME')
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize= [1,2,2,1], strides=[1,2,2,1], padding= 'SAME')
    L1 = tf.nn.dropout(L1, keep_prob=keep_prob)
#conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)
#pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], padding='SAME', strides=2)
#dropout1 = tf.layers.dropout(inputs=pool1, rate=0.7, training=self.training)

# Conv layer 2
# L2 ImgIn shape = (?, 14, 14, 32)
with tf.name_scope('ConvLayer2'):
    W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev= 0.01))
    # Conv   ->    (?,14,14,64)
    # Pool   ->    (?,7,7,64)
    L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides= [1,2,2,1], padding='SAME')
    L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

with tf.name_scope('ConvLayer3'):
    W3 = tf.Variable(tf.random_normal([3,3,64,128], stddev= 0.01))
    # Conv   ->    (?,7,7,128)
    # Pool   ->    (?,4,4,128)
    # Reshape ->   (?,4 * 4 * 128) Flatten them for FC
    L3 = tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding='SAME')
    L3 = tf.nn.relu(L3)
    L3 = tf.nn.max_pool(L3, ksize=[1,2,2,1], strides= [1,2,2,1], padding='SAME')
    L3 = tf.reshape(L3, [-1,4 * 4 * 128])
    L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
#conv3 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)
#....
#flat =tf.reshape(dropout3, [-1,128 *4 *4])
#dense4 = tf.layers.dense(inputs=flat, units=625, activation=tf,nn.relu)
#dropout4 = tf.layers.dropout(inputs=dense4, rate=0.5, training=self.training)

#Fully Connected layer1
#FC 4X4X128 inputs -> 625 outputs
with tf.name_scope('Fully_connected1'):
    W4 = tf.get_variable("W4", shape = [4 * 4 * 128,625],initializer = tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([625]))
    L4 = tf.nn.relu(tf.matmul(L3,W4) + b4)
    L4 = tf.nn.dropout(L4, keep_prob=keep_prob)
    
#Fully Connected layer2
#Final FC 625 inputs -> 10 outputs
with tf.name_scope('Fully_connected1'):
    W5 = tf.get_variable("W5", shape = [625,10],initializer = tf.contrib.layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([10]))
    hypothesis = tf.matmul(L4,W5) + b5
    hypothesis = tf.nn.dropout(hypothesis, keep_prob=keep_prob)
    
# cost/Loss func
with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
    
# optimizer    
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [32]:
# parameters
training_epochs = 15
batch_size = 100

# initialize
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs")
    writer.add_graph(sess.graph)

    # train my model
    print('Learning started. It takes sometime. ')
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feed_dict = {X:batch_xs, Y:batch_ys, keep_prob:0.7}
            c,_ = sess.run([cost, optimizer],feed_dict = feed_dict)
            avg_cost += c / total_batch
        print('Epoch:','%04d' %(epoch + 1), 'cost=','{:.9f}'.format(avg_cost))
    print('Learning Finished!')

    #Test model and check accuracy
    correct_prediction = tf.equal(tf.arg_max(hypothesis,1), tf.arg_max(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('Accuarcy:',sess.run(accuracy, feed_dict = {x:mnist.test.images, Y: mnist.test.labels, keep_prob:1}))

Learning started. It takes sometime. 


KeyboardInterrupt: 